In [18]:
import os

def connect_with_kaggle(kggle_user, kaggle_key, root_kaggle_path = '/root/.kaggle'):
    try:
        os.makedirs(root_kaggle_path) 
    except Exception as e:
        print(e)
    
    my_api = '{"username":"'+ kggle_user +'","key":"'+ kaggle_key +'"}'
    with open(f'{root_kaggle_path}/kaggle.json', 'w') as f:
        f.write(my_api)
        f.close()
    print(os.listdir(root_kaggle_path))

connect_with_kaggle(kggle_user='alialaa158', kaggle_key='f225f03630c43302918ad0fade2dd20a')

[Errno 17] File exists: '/root/.kaggle'
['kaggle.json']


In [2]:
!kaggle datasets download -d ahmedbenomar/street-imagery 

100% 3.73G/3.73G [02:14<00:00, 30.0MB/s]
100% 3.73G/3.73G [02:15<00:00, 29.7MB/s]


In [19]:
!kaggle kernels output alialaa158/notebookeb26cba0a8 -p ./out_train

Output file downloaded to ./out_train/notebook04602db644.log
Output file downloaded to ./out_train/notebookb58db029d6.log
Output file downloaded to ./out_train/notebookeec4544a10.log
Output file downloaded to ./out_train/train/exp/F1_curve.png
Output file downloaded to ./out_train/train/exp/PR_curve.png
Output file downloaded to ./out_train/train/exp/P_curve.png
Output file downloaded to ./out_train/train/exp/R_curve.png
Output file downloaded to ./out_train/train/exp/confusion_matrix.png
Output file downloaded to ./out_train/train/exp/events.out.tfevents.1674117827.b6215769a58a.278.0
Output file downloaded to ./out_train/train/exp/events.out.tfevents.1674161241.8ca8df326fc2.279.0
Output file downloaded to ./out_train/train/exp/events.out.tfevents.1674204868.e2f39b3f4a38.340.0
Output file downloaded to ./out_train/train/exp/events.out.tfevents.1674221685.ad5e52323589.222.0
Output file downloaded to ./out_train/train/exp/hyp.yaml
Output file downloaded to ./out_train/train/exp/labels.jp

In [4]:
!unzip -q street-imagery.zip -d street-imagery

In [5]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [6]:
%cd /content
!git clone https://github.com/m7mdmosa27/yolov5.git  # clone
%cd yolov5
!pip install -qr requirements.txt  # install

/content
Cloning into 'yolov5'...
remote: Enumerating objects: 11609, done.
remote: Total 11609 (delta 0), reused 0 (delta 0), pack-reused 11609
Receiving objects: 100% (11609/11609), 13.59 MiB | 15.30 MiB/s, done.
Resolving deltas: 100% (7692/7692), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 599.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.3 MB/s eta 0:00:00


In [20]:
!python detect.py --weights /content/out_train/train/exp/weights/best.pt --source /content/street-imagery/detect/images \
--save-txt --project '/content/detect/'

detect: weights=['/content/out_train/train/exp/weights/best.pt'], source=/content/street-imagery/detect/images, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=/content/detect/, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 da5bc5c Python-3.8.10 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 276 layers, 35306640 parameters, 0 gradients, 49.0 GFLOPs
image 1/2092 /content/street-imagery/detect/images/006e04a34f54d116f6aff2bb31cb320c.jpg: 384x640 (no detections), 25.4ms
image 2/2092 /content/street-imagery/detect/images/006e70d6a51af3e670bbe712a83aa1c5.jpg: 384x640 1 GRAFFITI, 25.2ms
image 3/2092 /content/street-imagery/detect/images/0081b25823f23a536c9f1952491db63

In [8]:
!cp '/content/gdrive/MyDrive/Smartathon/Theme #1/test.csv'   /content

In [21]:
import pandas as pd
import cv2
import numpy as np
from glob import glob
from random import randint
import os

def convert_y2xy(l, im_w, im_h):
    c, xc, yc, w,h = np.array(l.split(' '), np.float32)
    c, x1,y1, x2,y2  = np.array([c, (xc-(w/2))*im_w, (yc-(h/2))*im_h, (xc+(w/2))*im_w, (yc+(h/2))*im_h], np.float32)
    return c, (x1,y1), (x2,y2)


def create_csv_from_yolo():

    cls = {0: 'GRAFFITI', 1: 'FADED_SIGNAGE', 2: 'POTHOLES', 3: 'GARBAGE', 4: 'CONSTRUCTION_ROAD', 5: 'BROKEN_SIGNAGE',
           6: 'BAD_STREETLIGHT', 7: 'BAD_BILLBOARD', 8: 'SAND_ON_ROAD', 9: 'CLUTTER_SIDEWALK',10: 'UNKEPT_FACADE'}
    df = pd.DataFrame({}, columns =['class', 'image_path', 'name', 'xmax', 'xmin', 'ymax', 'ymin'])
        
    t_labels = glob('/content/detect/exp3/labels/*')
    df_test = pd.read_csv('/content/test.csv')['image_path'].to_list()
    p_imgs_path = '/content/detect/exp3'
    p_labels_path = '/content/detect/exp3/labels'


    for img_name in df_test:
        img_id = img_name.split('.')[0]
        id_flag = False

        try:
            with open(os.path.join(p_labels_path, f'{img_id}.txt'), 'r') as f:
                lines = f.readlines()
            
            img = cv2.imread(os.path.join(p_imgs_path, img_name))
            im_h, im_w = img.shape[:-1]
            df2 = []
            for line in lines:
                c, (x1,y1), (x2,y2) = convert_y2xy(line, im_w, im_h)
                c, (x1,y1), (x2,y2) =c, (x1/2,y1/2), (x2/2,y2/2) 
                df2.append({'class': c, 'image_path': img_name, 'name': cls[c],
                    'xmax': x2, 'xmin':x1, 'ymax':y2, 'ymin':y1})
            df = df.append(df2, ignore_index = True)
            print('Done With this label : ', img_id)

        except FileNotFoundError:
            print('label for this id : ', img_id,  ' is not exist so i will random it')
            df = df.append({'class': 3, 'image_path': img_name, 'name': 'GARBAGE',
                'xmax': randint(500,1000), 'xmin': randint(0,500),
                'ymax': randint(500,1000), 'ymin': randint(0,500)}, ignore_index = True)
        except Exception as e:
            print(e)
            raise
    return df

In [22]:
df = create_csv_from_yolo()
df.head()

Done With this label :  953ab1447c46ecfef67ab14629cd70c7
Done With this label :  e4ddbaa7970fca225a51288ce5f7d3f9
Done With this label :  5b8120d69607a077b7583334be3ba18b
label for this id :  138b1dc82005b4c33e4886260649d313  is not exist so i will random it
Done With this label :  0f91ec1533b845b13089f8cf4e0a36f7
Done With this label :  dfd51a42c03dc458daf2fbe071efee9d
Done With this label :  ab0e2122d7786186da2e1c4a3d3d6fb4
Done With this label :  51e45a98c72f86c44bc6340c6ca6a8e7
Done With this label :  443a50c860568eb0e6188525400f1736
Done With this label :  ea57cc594dba39b5b25617a7f6ae87d5
label for this id :  1697d2971efa752abd823083978cb3b1  is not exist so i will random it
label for this id :  3dea470248e8637636a0f89697b41236  is not exist so i will random it
Done With this label :  54bd134d15e7b274e3811b61257c12b9
Done With this label :  1745703a49f8870571667504fa93f585
Done With this label :  9f7b4611104b81e0c3e4f7e12dabe7ad
Done With this label :  0bd97eb285fcedd86932513f674b

,class,image_path,name,xmax,xmin,ymax,ymin
0,7.0,953ab1447c46ecfef67ab14629cd70c7.jpg,BAD_BILLBOARD,705.500061,612.000122,225.999985,168.000214
1,3.0,e4ddbaa7970fca225a51288ce5f7d3f9.jpg,GARBAGE,633.000000,507.000000,414.999969,329.500153
2,2.0,5b8120d69607a077b7583334be3ba18b.jpg,POTHOLES,411.499664,187.500000,527.000305,341.500031
3,2.0,5b8120d69607a077b7583334be3ba18b.jpg,POTHOLES,767.000183,525.499695,530.500305,340.000214
4,3.0,138b1dc82005b4c33e4886260649d313.jpg,GARBAGE,516.000000,340.000000,806.000000,139.000000


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3960 entries, 0 to 3959
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   class       3960 non-null   float64
 1   image_path  3960 non-null   object 
 2   name        3960 non-null   object 
 3   xmax        3960 non-null   float64
 4   xmin        3960 non-null   float64
 5   ymax        3960 non-null   float64
 6   ymin        3960 non-null   float64
dtypes: float64(5), object(2)
memory usage: 216.7+ KB


In [24]:
df.to_csv('/content/submit_file_v100.csv')

In [ ]:
%cd /content/street-imagery
!zip -r /content/detect.zip detect

/content/street-imagery
updating: detect/ (stored 0%)
  adding: detect/images/ (stored 0%)
  adding: detect/images/6d5cb9abca91a010f1936f3be369223b.jpg (deflated 2%)
  adding: detect/images/12262af4600e5b82b67cdfd43039550e.jpg (deflated 1%)
  adding: detect/images/519e6f8b6e54601f761506e7d16fd836.jpg (deflated 0%)
  adding: detect/images/1d920bc3f201f392a8ddfa5dbefa89e8.jpg (deflated 2%)
  adding: detect/images/d540a9a121a2c1e393a509dd267b4ec2.jpg (deflated 1%)
  adding: detect/images/904e69719e3e879676400a29c74f2ac0.jpg (deflated 0%)
  adding: detect/images/be20bfb23d30668ede124d229a1759ea.jpg (deflated 0%)
  adding: detect/images/7f2e5154b15c998b1c724acb2106fe07.jpg (deflated 1%)
  adding: detect/images/ed8848c4ce41d9ba70d6d1ab364094ed.jpg (deflated 3%)
  adding: detect/images/d06f7f75b7d004dfbb4093a624b1ebc5.jpg (deflated 3%)
  adding: detect/images/74c0771e2418ae6cd4c2ce2a5d3cde1b.jpg (deflated 3%)
  adding: detect/images/66848d6374bc8c801c4d2dd3da1e0ba0.jpg (deflated 1%)
  adding:

In [ ]:
!cp /content/detect.zip '/content/drive/MyDrive/Job_projects/For Fun/Competition'